In [10]:
#import wget
import pandas as pd
import numpy as np

# Run line below once
# wget.download("https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv")

## Data Preparation

In [11]:
cars = pd.read_csv("data.csv")
features = ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']
filtered_data = cars[features]

filtered_data.columns = filtered_data.columns.str.replace(' ', '_').str.lower()
filtered_data.rename(columns={'msrp': 'price'}, inplace=True)

filtered_data['engine_hp'] = filtered_data['engine_hp'].fillna(0)
filtered_data['engine_cylinders'] = filtered_data['engine_cylinders'].fillna(0)

filtered_data.isna().sum()
filtered_data

C:\Users\HP\AppData\Local\Temp\ipykernel_31668\2304709633.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.rename(columns={'msrp': 'price'}, inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_31668\2304709633.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['engine_hp'] = filtered_data['engine_hp'].fillna(0)
C:\Users\HP\AppData\Local\Temp\ipykernel_31668\2304709633.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


## Question 1

In [12]:
filtered_data['transmission_type'].mode().values[0]

'AUTOMATIC'

## Question 2

In [13]:
correlation_matrix = filtered_data.corr()
mean_price = filtered_data['price'].mean()
filtered_data['above_average'] = (filtered_data['price'] > mean_price).astype(int)

from sklearn.model_selection import train_test_split

X = filtered_data.drop(columns=['price', 'above_average'])
y = filtered_data['above_average']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


C:\Users\HP\AppData\Local\Temp\ipykernel_31668\2294687670.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = filtered_data.corr()
C:\Users\HP\AppData\Local\Temp\ipykernel_31668\2294687670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['above_average'] = (filtered_data['price'] > mean_price).astype(int)


## Question 3

In [14]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

categorical_features = ['make', 'model', 'transmission_type', 'vehicle_style']
label_encoders = {}
mutual_info_scores = []

for feature in categorical_features:
    le = LabelEncoder()
    X_encoded = le.fit_transform(X_train[feature])
    mi_score = mutual_info_classif(X_encoded.reshape(-1, 1), y_train, discrete_features=True, random_state=42)
    mutual_info_scores.append(mi_score[0])
    label_encoders[feature] = le

feature_scores = dict(zip(categorical_features, mutual_info_scores))

lowest_score_variable = min(feature_scores, key=feature_scores.get)

rounded_scores = {k: round(v, 2) for k, v in feature_scores.items()}

print("Mutual Information Scores:")
for feature, score in rounded_scores.items():
    print(f"{feature}: {score}")

print(f"{lowest_score_variable}")


Mutual Information Scores:
make: 0.24
model: 0.46
transmission_type: 0.02
vehicle_style: 0.08
transmission_type


## Question 4

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

categorical_features = ['make', 'model', 'transmission_type', 'vehicle_style']

preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'
)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)

accuracy = round(accuracy_score(y_val, y_pred), 2)

print(f"{accuracy}")


0.95


## Question 5

## Question 6

In [16]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

numeric_features = X_train.select_dtypes(include=[np.number])

y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

alphas = [0, 0.01, 0.1, 1, 10]

best_alpha = None
best_rmse = float('inf')

for alpha in alphas:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(numeric_features, y_train_log)

    X_val_numeric = X_val.select_dtypes(include=[np.number])
    y_pred_log = model.predict(X_val_numeric)

    rmse = round(sqrt(mean_squared_error(y_val_log, y_pred_log)), 3)

    if rmse < best_rmse:
        best_rmse = rmse
        best_alpha = alpha

print(f"{best_alpha}")


0
